In [15]:
import pandas as pd
import numpy as np

data = pd.read_csv("/home/aa/Big2025/ml_dl_python/data/titanic/train.csv")
print(data.info())
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                      

In [ ]:
df = data[["Pclass", "Age", "SibSp", "Fare", "Survived"]]
df.dropna(inplace=True)
df_input = df.iloc[:, :-1].values
df_target = df.iloc[:, -1].values
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_input, df_target, test_size=0.2)
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method="hist")
xgb.fit(X_train, y_train)

from sklearn.metrics import accuracy_score

y_pred = xgb.predict(X_test)
accurate = accuracy_score(y_test, y_pred)
print(accurate)

/tmp/ipykernel_259690/2230385496.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


0.6573426573426573


In [17]:
test = pd.read_csv("/home/aa/Big2025/ml_dl_python/data/titanic/test.csv")

test_df = test[["Pclass", "Age", "SibSp", "Fare"]]

print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB
None


In [18]:
pred_test = xgb.predict(test_df)
pred_test.astype(int)
submisstion = pd.DataFrame({"PassengerId": test["PassengerId"], "Survived": pred_test})

In [19]:
submisstion.info()
submisstion.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  418 non-null    int64
 1   Survived     418 non-null    int64
dtypes: int64(2)
memory usage: 6.7 KB


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [20]:
submisstion.to_csv("/home/aa/Big2025/ml_dl_python/data/titanic/xgb_titanic.csv", index=False)

In [ ]:
%pip install seaborn

In [37]:
# [1] 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

# [2] 데이터 불러오기
train = pd.read_csv("/home/aa/Big2025/ml_dl_python/data/titanic/train.csv")
test = pd.read_csv("/home/aa/Big2025/ml_dl_python/data/titanic/test.csv")

# [3] 결측치 처리
train["Age"].fillna(train["Age"].mean(), inplace=True)
test["Age"].fillna(train["Age"].mean(), inplace=True)

train["Embarked"].fillna(train["Embarked"].mode()[0], inplace=True)
test["Embarked"].fillna(train["Embarked"].mode()[0], inplace=True)

test["Fare"].fillna(train["Fare"].mean(), inplace=True)

# [4] 범주형 인코딩
le_sex = LabelEncoder()
le_embarked = LabelEncoder()

train["Sex"] = le_sex.fit_transform(train["Sex"])
test["Sex"] = le_sex.transform(test["Sex"])

train["Embarked"] = le_embarked.fit_transform(train["Embarked"])
test["Embarked"] = le_embarked.transform(test["Embarked"])

# [5] 학습 데이터 구성
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
X = train[features]
y = train["Survived"]
X_final_test = test[features]

# [6] 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# [7] 하이퍼파라미터 튜닝 (GridSearch)
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

xgb_base = XGBClassifier(tree_method="hist", use_label_encoder=False, eval_metric="logloss")
grid_search = GridSearchCV(estimator=xgb_base, param_grid=param_grid, cv=3, scoring='accuracy', verbose=0)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

# [8] 성능 확인 (검증 정확도)
y_pred_val = best_model.predict(X_val)
print("검증 정확도:", accuracy_score(y_val, y_pred_val))

# [9] 테스트셋 예측 및 제출 파일 생성
pred_final = best_model.predict(X_final_test).astype(int)

submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": pred_final
})

# [10] 행 수 확인 및 저장
assert submission.shape[0] == 418, "⚠️ 제출 파일은 418행이어야 합니다."
submission.to_csv("/home/aa/Big2025/ml_dl_python/data/titanic/parick.csv", index=False)


/tmp/ipykernel_259690/2485738533.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train["Age"].fillna(train["Age"].mean(), inplace=True)
/tmp/ipykernel_259690/2485738533.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=

검증 정확도: 0.8156424581005587


/home/aa/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [15:03:46] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/aa/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [15:03:47] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# 데이터 불러오기
train = pd.read_csv("/home/aa/Big2025/ml_dl_python/data/titanic/train.csv")
test = pd.read_csv("/home/aa/Big2025/ml_dl_python/data/titanic/test.csv")

# [1] 호칭 추출
def extract_title(name):
    title = name.split(',')[1].split('.')[0].strip()
    return title

train["Title"] = train["Name"].apply(extract_title)
test["Title"] = test["Name"].apply(extract_title)

# 희소 호칭 병합
for dataset in [train, test]:
    dataset["Title"] = dataset["Title"].replace(['Lady', 'Countess', 'Capt', 'Col',
        'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset["Title"] = dataset["Title"].replace(['Mlle', 'Ms'], 'Miss')
    dataset["Title"] = dataset["Title"].replace('Mme', 'Mrs')

# [2] 가족 유무 파생 변수
train["FamilySize"] = train["SibSp"] + train["Parch"]
test["FamilySize"] = test["SibSp"] + test["Parch"]
train["IsAlone"] = (train["FamilySize"] == 0).astype(int)
test["IsAlone"] = (test["FamilySize"] == 0).astype(int)

# [3] 결측치 처리
train["Age"].fillna(train["Age"].median(), inplace=True)
test["Age"].fillna(train["Age"].median(), inplace=True)
train["Embarked"].fillna(train["Embarked"].mode()[0], inplace=True)
test["Embarked"].fillna(train["Embarked"].mode()[0], inplace=True)
test["Fare"].fillna(train["Fare"].median(), inplace=True)

# [4] 범주형 One-Hot 인코딩
combined = pd.concat([train, test], sort=False)
combined = pd.get_dummies(combined, columns=["Sex", "Embarked", "Title"], drop_first=True)

# 다시 분리
train_encoded = combined[:len(train)]
test_encoded = combined[len(train):]

# [5] 학습/테스트 구성
features = ["Pclass", "Age", "Fare", "FamilySize", "IsAlone"] + \
           [col for col in combined.columns if col.startswith(("Sex_", "Embarked_", "Title_"))]

X = train_encoded[features]
y = train["Survived"]
X_test_final = test_encoded[features]

# [6] 학습/검증 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# [7] 모델 앙상블 (Voting)
xgb_clf = XGBClassifier(n_estimators=150, max_depth=5, learning_rate=0.1,
                        subsample=0.9, colsample_bytree=0.8,
                        tree_method="hist", use_label_encoder=False, eval_metric="logloss")

rf_clf = RandomForestClassifier(n_estimators=200, max_depth=7, random_state=42)
lr_clf = LogisticRegression(max_iter=200)

voting_clf = VotingClassifier(estimators=[
    ('xgb', xgb_clf),
    ('rf', rf_clf),
    ('lr', lr_clf)
], voting='soft')

voting_clf.fit(X_train, y_train)
y_pred = voting_clf.predict(X_val)

# [8] 성능 출력
acc = accuracy_score(y_val, y_pred)
print(f"개선된 검증 정확도: {acc:.5f}")


/tmp/ipykernel_259690/664195175.py:36: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train["Age"].fillna(train["Age"].median(), inplace=True)
/tmp/ipykernel_259690/664195175.py:37: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=

개선된 검증 정확도: 0.81564


/home/aa/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [39]:
%pip install catboost

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 50.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.0/359.0 KB 29.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [41]:
# 타이타닉 생존자 예측 - CatBoost 모델 기반 개선
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [48]:
# 데이터 불러오기
train = pd.read_csv("/home/aa/Big2025/ml_dl_python/data/titanic/train.csv")
test = pd.read_csv("/home/aa/Big2025/ml_dl_python/data/titanic/test.csv")


In [49]:
# Title 파생 변수 생성 및 정제
train["Title"] = train["Name"].str.extract(' ([A-Za-z]+)\\.', expand=False)
test["Title"] = test["Name"].str.extract(' ([A-Za-z]+)\\.', expand=False)
title_mapping = {
    "Mr": "Mr", "Miss": "Miss", "Mrs": "Mrs", "Master": "Master",
    "Dr": "Rare", "Rev": "Rare", "Col": "Rare", "Major": "Rare",
    "Mlle": "Miss", "Countess": "Rare", "Ms": "Miss", "Lady": "Rare",
    "Jonkheer": "Rare", "Don": "Rare", "Dona": "Rare", "Mme": "Mrs", "Capt": "Rare", "Sir": "Rare"
}
train["Title"] = train["Title"].map(title_mapping).fillna("Rare")
test["Title"] = test["Title"].map(title_mapping).fillna("Rare")

In [50]:
# FamilySize, IsAlone 변수 생성
for df in [train, test]:
    df["FamilySize"] = df["SibSp"] + df["Parch"]
    df["IsAlone"] = (df["FamilySize"] == 0).astype(int)

# Cabin 앞글자 → Deck
train["Deck"] = train["Cabin"].astype(str).str[0].replace("n", "Unknown")
test["Deck"] = test["Cabin"].astype(str).str[0].replace("n", "Unknown")

In [51]:
# 결측치 처리
train["Age"].fillna(train["Age"].median(), inplace=True)
test["Age"].fillna(train["Age"].median(), inplace=True)
train["Fare"].fillna(train["Fare"].median(), inplace=True)
test["Fare"].fillna(train["Fare"].median(), inplace=True)
train["Embarked"].fillna(train["Embarked"].mode()[0], inplace=True)
test["Embarked"].fillna(train["Embarked"].mode()[0], inplace=True)

/tmp/ipykernel_259690/3360460720.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train["Age"].fillna(train["Age"].median(), inplace=True)
/tmp/ipykernel_259690/3360460720.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=

In [52]:
# 학습 준비
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare",
            "Embarked", "Title", "FamilySize", "IsAlone", "Deck"]
X = train[features]
y = train["Survived"]
X_test_final = test[features]
cat_features = ["Sex", "Embarked", "Title", "Deck"]

In [53]:
# 학습 및 평가
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
model = CatBoostClassifier(iterations=500, learning_rate=0.05, depth=6,
                           cat_features=cat_features, verbose=0)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
print("검증 정확도:", accuracy_score(y_val, y_pred))

검증 정확도: 0.8156424581005587


In [54]:
# 예측 및 결과 저장
y_test_pred = model.predict(X_test_final).astype(int)
submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": y_test_pred
})
submission.to_csv("/home/aa/Big2025/ml_dl_python/data/titanic/parick.csv", index=False)
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
